In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F  
import torchvision
from torchvision import models
import numpy as np
import pandas as pd
from PIL import Image
import os
import shutil
import collections
import pickle

In [2]:
torch.__version__

'1.2.0'

In [3]:
device = "cpu"

In [4]:
attack_with_NN_results_dir = "attack_with_NN_resnet50_feature_shapes"
if not os.path.exists(attack_with_NN_results_dir):
    os.makedirs(attack_with_NN_results_dir)

In [5]:
def get_transform(device):
    """
    Input: numpy.ndarray, shape = (28, 28), uint8
    Output: torch.Tensor (cpu or cuda), torch.Size([1, 3, 224, 224]), torch.float32
    """
    transform = torchvision.transforms.Compose(
        [lambda x: Image.fromarray(x),
         lambda x: x.convert("RGB") if x.mode == "L" else x,
         torchvision.transforms.Resize([224, 224]),
         torchvision.transforms.ToTensor(),
         torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225]),
         lambda x: torch.unsqueeze(x, 0),
         lambda x: x.to(device)],
    )
    return transform

# preprocessor
transform = get_transform(device)

# define resnet50
model = models.resnet50(pretrained=False)
fc_in_features = model.fc.in_features
model.fc = nn.Linear(fc_in_features, 10)

# input
random_input = np.random.randint(low=0, high=256, size=(28, 28), dtype=np.uint8)
random_input = transform(random_input)
print(type(random_input), random_input.shape, random_input.dtype)

y = torch.from_numpy(np.random.randint(low=0, high=10, size=(1), dtype=np.int64))
print(type(y), y.shape, y.dtype)

# forward pass
model.train()
output = model(random_input)
print(type(output), output.shape, output.dtype, output)

# backward pass
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(output, y)
loss.backward()
print(type(loss), loss.shape, loss.dtype, loss.item())


print("OK")

<class 'torch.Tensor'> torch.Size([1, 3, 224, 224]) torch.float32
<class 'torch.Tensor'> torch.Size([1]) torch.int64
<class 'torch.Tensor'> torch.Size([1, 10]) torch.float32 tensor([[ 0.3862,  0.4995,  0.4248, -0.3419,  0.4738,  0.2802, -0.2655,  0.4078,
         -1.5438,  0.4691]], grad_fn=<AddmmBackward>)
<class 'torch.Tensor'> torch.Size([]) torch.float32 2.8573009967803955
OK


In [6]:
tmp = []
for name, layer in model.named_modules():
    tmp.append(type(layer))
tmp = list(set(tmp))
tmp

[torch.nn.modules.conv.Conv2d,
 torch.nn.modules.batchnorm.BatchNorm2d,
 torchvision.models.resnet.ResNet,
 torch.nn.modules.container.Sequential,
 torch.nn.modules.linear.Linear,
 torchvision.models.resnet.Bottleneck,
 torch.nn.modules.activation.ReLU,
 torch.nn.modules.pooling.MaxPool2d,
 torch.nn.modules.pooling.AdaptiveAvgPool2d]

In [7]:
cnt = 0
for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.modules.conv.Conv2d):
        cnt += 1
        print(cnt, name)

1 conv1
2 layer1.0.conv1
3 layer1.0.conv2
4 layer1.0.conv3
5 layer1.0.downsample.0
6 layer1.1.conv1
7 layer1.1.conv2
8 layer1.1.conv3
9 layer1.2.conv1
10 layer1.2.conv2
11 layer1.2.conv3
12 layer2.0.conv1
13 layer2.0.conv2
14 layer2.0.conv3
15 layer2.0.downsample.0
16 layer2.1.conv1
17 layer2.1.conv2
18 layer2.1.conv3
19 layer2.2.conv1
20 layer2.2.conv2
21 layer2.2.conv3
22 layer2.3.conv1
23 layer2.3.conv2
24 layer2.3.conv3
25 layer3.0.conv1
26 layer3.0.conv2
27 layer3.0.conv3
28 layer3.0.downsample.0
29 layer3.1.conv1
30 layer3.1.conv2
31 layer3.1.conv3
32 layer3.2.conv1
33 layer3.2.conv2
34 layer3.2.conv3
35 layer3.3.conv1
36 layer3.3.conv2
37 layer3.3.conv3
38 layer3.4.conv1
39 layer3.4.conv2
40 layer3.4.conv3
41 layer3.5.conv1
42 layer3.5.conv2
43 layer3.5.conv3
44 layer4.0.conv1
45 layer4.0.conv2
46 layer4.0.conv3
47 layer4.0.downsample.0
48 layer4.1.conv1
49 layer4.1.conv2
50 layer4.1.conv3
51 layer4.2.conv1
52 layer4.2.conv2
53 layer4.2.conv3


In [8]:
cnt = 0
for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.modules.activation.ReLU):
        cnt += 1
        print(cnt, name)

1 relu
2 layer1.0.relu
3 layer1.1.relu
4 layer1.2.relu
5 layer2.0.relu
6 layer2.1.relu
7 layer2.2.relu
8 layer2.3.relu
9 layer3.0.relu
10 layer3.1.relu
11 layer3.2.relu
12 layer3.3.relu
13 layer3.4.relu
14 layer3.5.relu
15 layer4.0.relu
16 layer4.1.relu
17 layer4.2.relu


In [9]:
cnt = 0
for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.modules.linear.Linear):
        cnt += 1
        print(cnt, name)

1 fc


In [10]:
# define resnet50
model = models.resnet50(pretrained=False)
fc_in_features = model.fc.in_features
model.fc = nn.Linear(fc_in_features, 10)

# hooks for hidden layer intermediate features
hidden_layer_features = {}
def get_hidden_layer_features(name):
    def hook(model, input, output):
        hidden_layer_features[name] = output.detach().view(-1)
    return hook

for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.modules.conv.Conv2d):
        layer.register_forward_hook(get_hidden_layer_features(name))
        

random_input = np.random.randint(low=0, high=256, size=(28, 28), dtype=np.uint8)
random_input = transform(random_input)
y = torch.from_numpy(np.random.randint(low=0, high=10, size=(1), dtype=np.int64))
output = model(random_input)

print(type(hidden_layer_features), len(hidden_layer_features))
for k in hidden_layer_features.keys():
    print(type(hidden_layer_features[k]), hidden_layer_features[k].shape, len(hidden_layer_features[k]))
    
total = 0
for k in hidden_layer_features.keys():
    total += len(hidden_layer_features[k])
print(total)

<class 'dict'> 53
<class 'torch.Tensor'> torch.Size([802816]) 802816
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([802816]) 802816
<class 'torch.Tensor'> torch.Size([802816]) 802816
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([802816]) 802816
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([802816]) 802816
<class 'torch.Tensor'> torch.Size([401408]) 401408
<class 'torch.Tensor'> torch.Size([100352]) 100352
<class 'torch.Tensor'> torch.Size([401408]) 401408
<class 'torch.Tensor'> torch.Size([401408]) 401408
<class 'torch.Tensor'> torch.Size([100352]) 100352
<class 'torch.Tensor'> torch.Size([100352]) 100352
<class 'torch.Tensor'> torch.Size([401408]) 401408
<class 'torch.Tensor'> torch.Size([100352]) 100352
<class 'torch

In [11]:
# define resnet50
model = models.resnet50(pretrained=False)
fc_in_features = model.fc.in_features
model.fc = nn.Linear(fc_in_features, 10)

# hooks for hidden layer intermediate features
hidden_layer_features = {}
def get_hidden_layer_features(name):
    def hook(model, input, output):
        hidden_layer_features[name] = output.detach()
    return hook

for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.modules.activation.ReLU):
        layer.register_forward_hook(get_hidden_layer_features(name))
        

random_input = np.random.randint(low=0, high=256, size=(28, 28), dtype=np.uint8)
random_input = transform(random_input)
y = torch.from_numpy(np.random.randint(low=0, high=10, size=(1), dtype=np.int64))
output = model(random_input)

print(type(hidden_layer_features), len(hidden_layer_features))
for k in hidden_layer_features.keys():
    print(type(hidden_layer_features[k]), hidden_layer_features[k].shape, hidden_layer_features[k].dtype)
    
total = 0
for k in hidden_layer_features.keys():
    total += len(hidden_layer_features[k])
print(total)

<class 'dict'> 17
<class 'torch.Tensor'> torch.Size([1, 64, 112, 112]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 256, 56, 56]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 256, 56, 56]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 256, 56, 56]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 512, 28, 28]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 512, 28, 28]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 512, 28, 28]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 512, 28, 28]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 1024, 14, 14]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 1024, 14, 14]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 1024, 14, 14]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 1024, 14, 14]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 1024, 14, 14]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 1024, 14, 14]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 2048, 7, 7]) 

In [12]:
# define resnet50
model = models.resnet50(pretrained=False)
fc_in_features = model.fc.in_features
model.fc = nn.Linear(fc_in_features, 10)

# hooks for hidden layer intermediate features
hidden_layer_features = {}
def get_hidden_layer_features(name):
    def hook(model, input, output):
        hidden_layer_features[name] = output.detach().view(-1)
    return hook

for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.modules.activation.ReLU):
        layer.register_forward_hook(get_hidden_layer_features(name))
        

random_input = np.random.randint(low=0, high=256, size=(28, 28), dtype=np.uint8)
random_input = transform(random_input)
y = torch.from_numpy(np.random.randint(low=0, high=10, size=(1), dtype=np.int64))
output = model(random_input)

print(type(hidden_layer_features), len(hidden_layer_features))
for k in hidden_layer_features.keys():
    print(type(hidden_layer_features[k]), hidden_layer_features[k].shape, len(hidden_layer_features[k]))
    
total = 0
for k in hidden_layer_features.keys():
    total += len(hidden_layer_features[k])
print(total)

<class 'dict'> 17
<class 'torch.Tensor'> torch.Size([802816]) 802816
<class 'torch.Tensor'> torch.Size([802816]) 802816
<class 'torch.Tensor'> torch.Size([802816]) 802816
<class 'torch.Tensor'> torch.Size([802816]) 802816
<class 'torch.Tensor'> torch.Size([401408]) 401408
<class 'torch.Tensor'> torch.Size([401408]) 401408
<class 'torch.Tensor'> torch.Size([401408]) 401408
<class 'torch.Tensor'> torch.Size([401408]) 401408
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([200704]) 200704
<class 'torch.Tensor'> torch.Size([100352]) 100352
<class 'torch.Tensor'> torch.Size([100352]) 100352
<class 'torch.Tensor'> torch.Size([100352]) 100352
6322176


In [13]:
# define resnet50
model = models.resnet50(pretrained=False)
fc_in_features = model.fc.in_features
model.fc = nn.Linear(fc_in_features, 10)

# hooks for hidden layer intermediate features
hidden_layer_features = {}
def get_hidden_layer_features(name):
    def hook(model, input, output):
        hidden_layer_features[name] = output.detach().squeeze(dim=0)
    return hook

for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.modules.conv.Conv2d):
        layer.register_forward_hook(get_hidden_layer_features(name))
        

random_input = np.random.randint(low=0, high=256, size=(28, 28), dtype=np.uint8)
random_input = transform(random_input)
y = torch.from_numpy(np.random.randint(low=0, high=10, size=(1), dtype=np.int64))
output = model(random_input)

hidden_layer_features_shape = collections.OrderedDict()
print(type(hidden_layer_features), len(hidden_layer_features))
for k in hidden_layer_features.keys():
    print(k, type(hidden_layer_features[k]), hidden_layer_features[k].shape, hidden_layer_features[k].dtype)
    hidden_layer_features_shape[k] = np.array(hidden_layer_features[k].shape).tolist()
    
print()
total = 0
for k in hidden_layer_features.keys():
    total += len(hidden_layer_features[k])
print(total)
print()

print(hidden_layer_features_shape)
with open(os.path.join(attack_with_NN_results_dir, "hidden_layer_features_shape.pkl"), "wb") as f:
    pickle.dump(hidden_layer_features_shape, f)

<class 'dict'> 53
conv1 <class 'torch.Tensor'> torch.Size([64, 112, 112]) torch.float32
layer1.0.conv1 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.0.conv2 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.0.conv3 <class 'torch.Tensor'> torch.Size([256, 56, 56]) torch.float32
layer1.0.downsample.0 <class 'torch.Tensor'> torch.Size([256, 56, 56]) torch.float32
layer1.1.conv1 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.1.conv2 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.1.conv3 <class 'torch.Tensor'> torch.Size([256, 56, 56]) torch.float32
layer1.2.conv1 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.2.conv2 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.2.conv3 <class 'torch.Tensor'> torch.Size([256, 56, 56]) torch.float32
layer2.0.conv1 <class 'torch.Tensor'> torch.Size([128, 56, 56]) torch.float32
layer2.0.conv2 <class 'torch.Tensor'> torch.Size([128

In [14]:
# define resnet50
model = models.resnet50(pretrained=False)
fc_in_features = model.fc.in_features
model.fc = nn.Linear(fc_in_features, 10)

# random input
random_input = np.random.randint(low=0, high=256, size=(28, 28), dtype=np.uint8)
random_input = transform(random_input)
y = torch.from_numpy(np.random.randint(low=0, high=10, size=(1), dtype=np.int64))

# optimizer 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# forward pass
model.train()
logits = model(random_input)

# backward pass
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(logits, y)
print(type(loss), loss.shape, loss.dtype, loss.item())

optimizer.zero_grad()
loss.backward()

param_gradients_shape = collections.OrderedDict()
for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.modules.conv.Conv2d):
        param_gradients_shape[name] = np.array(layer.weight.grad.shape).tolist()
        print(name, layer.weight.grad.shape, layer.weight.grad.dtype)
optimizer.step()

print(param_gradients_shape)

with open(os.path.join(attack_with_NN_results_dir, "param_gradients_shape.pkl"), "wb") as f:
    pickle.dump(param_gradients_shape, f)

<class 'torch.Tensor'> torch.Size([]) torch.float32 2.6498537063598633
conv1 torch.Size([64, 3, 7, 7]) torch.float32
layer1.0.conv1 torch.Size([64, 64, 1, 1]) torch.float32
layer1.0.conv2 torch.Size([64, 64, 3, 3]) torch.float32
layer1.0.conv3 torch.Size([256, 64, 1, 1]) torch.float32
layer1.0.downsample.0 torch.Size([256, 64, 1, 1]) torch.float32
layer1.1.conv1 torch.Size([64, 256, 1, 1]) torch.float32
layer1.1.conv2 torch.Size([64, 64, 3, 3]) torch.float32
layer1.1.conv3 torch.Size([256, 64, 1, 1]) torch.float32
layer1.2.conv1 torch.Size([64, 256, 1, 1]) torch.float32
layer1.2.conv2 torch.Size([64, 64, 3, 3]) torch.float32
layer1.2.conv3 torch.Size([256, 64, 1, 1]) torch.float32
layer2.0.conv1 torch.Size([128, 256, 1, 1]) torch.float32
layer2.0.conv2 torch.Size([128, 128, 3, 3]) torch.float32
layer2.0.conv3 torch.Size([512, 128, 1, 1]) torch.float32
layer2.0.downsample.0 torch.Size([512, 256, 1, 1]) torch.float32
layer2.1.conv1 torch.Size([128, 512, 1, 1]) torch.float32
layer2.1.conv

## Record hidden layer features and gradients in one forward pass and one backward pass

In [17]:
class HiddenLayerFeatures:
    """
    one object for each data point (x, y).
    
    If there are 2*N data points (defender+reserve), 
    then 2*N such objects need to be instantiated. 
    """
    def __init__(self, idx):
        self.idx = idx
        self.data = {}
        
    def get_hook(self, name):
        def hook(model, input, output):
            # .squeeze(dim=0) because of batch_size = 1
            self.data[name] = output.detach().squeeze(dim=0)
        return hook


# random input
random_input = np.random.randint(low=0, high=256, size=(28, 28), dtype=np.uint8)
random_input = transform(random_input)
print(type(random_input), random_input.shape, random_input.dtype)
y = torch.from_numpy(np.random.randint(low=0, high=10, size=(1), dtype=np.int64))
print(type(y), y.shape, y.dtype)
print()

# define resnet50
model = models.resnet50(pretrained=False)
fc_in_features = model.fc.in_features
model.fc = nn.Linear(fc_in_features, 10)

# optimizer 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


# set up forward_hooks
hidden_layer_features = HiddenLayerFeatures(0)
for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.modules.conv.Conv2d):
        layer.register_forward_hook(hidden_layer_features.get_hook(name))


# forward pass
model.train()
logits = model(random_input)

# get hidden layer features
for k in hidden_layer_features.data.keys():
    print(k, type(hidden_layer_features.data[k]), hidden_layer_features.data[k].shape, 
          hidden_layer_features.data[k].dtype)

print()

criterion = torch.nn.CrossEntropyLoss()
loss = criterion(logits, y)
print(type(loss), loss.shape, loss.dtype, loss.item())

print()

# backward pass
optimizer.zero_grad()
loss.backward()

for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.modules.conv.Conv2d):
        print(name, layer.weight.grad.shape, layer.weight.grad.dtype)

#optimizer.step()

<class 'torch.Tensor'> torch.Size([1, 3, 224, 224]) torch.float32
<class 'torch.Tensor'> torch.Size([1]) torch.int64

conv1 <class 'torch.Tensor'> torch.Size([64, 112, 112]) torch.float32
layer1.0.conv1 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.0.conv2 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.0.conv3 <class 'torch.Tensor'> torch.Size([256, 56, 56]) torch.float32
layer1.0.downsample.0 <class 'torch.Tensor'> torch.Size([256, 56, 56]) torch.float32
layer1.1.conv1 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.1.conv2 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.1.conv3 <class 'torch.Tensor'> torch.Size([256, 56, 56]) torch.float32
layer1.2.conv1 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.2.conv2 <class 'torch.Tensor'> torch.Size([64, 56, 56]) torch.float32
layer1.2.conv3 <class 'torch.Tensor'> torch.Size([256, 56, 56]) torch.float32
layer2.0.conv1 <class 'torch.Te